Decorators are functions which modify the functionality of another function. They help to make our code shorter and more Pythonic. 

In [6]:
def decorator_func(func):
    def wrapperFunc():
        print("Before calling func()")
        func()
        print("After calling func()")

    return wrapperFunc


def function_needs_decoration():
    print("Executing function_needs_decoration")

function_needs_decoration()
print("")

function_needs_decoration = decorator_func(function_needs_decoration)

function_needs_decoration()

Executing function_needs_decoration

Before calling func()
Executing function_needs_decoration
After calling func()


In [9]:
# Using standard decorator code style
# @decorator_func   - is short way for 
# function_needs_decoration = decorator_func(function_needs_decoration)

@decorator_func
def function_needs_decoration():
    print("Executing function_needs_decoration")

function_needs_decoration()
print("")

print(function_needs_decoration.__name__)

Before calling func()
Executing function_needs_decoration
After calling func()

wrapperFunc


In [10]:
from functools import wraps
def decorator_func(func):
    @wraps(func)
    def wrapperFunc():
        print("Before calling func()")
        func()
        print("After calling func()")

    return wrapperFunc

@decorator_func
def function_needs_decoration():
    print("Executing function_needs_decoration")

function_needs_decoration()
print("")

print(function_needs_decoration.__name__)

Before calling func()
Executing function_needs_decoration
After calling func()

function_needs_decoration


In [15]:
from functools import wraps
def decorator_name(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        if not can_run:
            return "Function will not run"
        return f(*args, **kwargs)
    return decorated

@decorator_name
def func():
    return("Function is running")

can_run = True
print(func())

can_run = False
print(func())

Function is running
Function will not run


In [14]:
# Loging example

def logit(func):
    @wraps(func)
    def with_logging(*args, **kwargs):
        print(func.__name__ + " was called")
        return func(*args, **kwargs)
    return with_logging

@logit
def addition_func(x):
   return x + x


result = addition_func(4)
print(result)

addition_func was called
8


In [47]:
# Decorators with Arguments
# 1. Nesting a Decorator Within a Function

def logit(logfile='tmp/out.log'):
    def logging_decorator(func):
        @wraps(func)
        def wrapped_function(*args, **kwargs):
            log_string = func.__name__ + " was called"
            print(log_string)
            # Open the logfile and append
            with open(logfile, 'a') as opened_file:
                # Now we log to the specified logfile
                opened_file.write(log_string + '\n')
        return wrapped_function
    return logging_decorator

@logit()
def myfunc1():
    pass

myfunc1()

@logit(logfile='tmp/func2.log')
def myfunc2():
    pass

myfunc2()


myfunc1 was called
myfunc2 was called


In [34]:
import time
def timeit(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        t = time.clock()
        res = func(*args, **kwargs)
        print("Time taken by", func.__name__, "=", time.clock()-t)
        return res

    return wrapper

@timeit
def myFunction():
    for i in range(20000000):
        pass

myFunction()

Time taken by myFunction = 0.4987671923961443


## Useful Decorators
### lru_cache
For caching the output of a function for given set of arguments.  
lru_cache - Replace Least Recently Used Cached results with new one.  
if maxsize=None, then remember results of all previous calls.  
if maxsize=10, then remember 10 most recent unique results.  


In [49]:
from functools import lru_cache
@lru_cache(maxsize=2)
def summ(a, b):
    print("Called with args a = ", a, " b = ", b)
    return a + b

print(summ(1, 1))
print(summ(1, 2))
# Following are cached
print(summ(1, 1))
print(summ(1, 2))
# Following call replaces cache with args (1, 1)
print(summ(2, 1))
# This will call the function
print(summ(1, 1))

# We can clear cache
summ.cache_clear()
# This will call the function
print(summ(1, 1))
print(summ(2, 1))


Called with args a =  1  b =  1
2
Called with args a =  1  b =  2
3
2
3
Called with args a =  2  b =  1
3
Called with args a =  1  b =  1
2
Called with args a =  1  b =  1
2
Called with args a =  2  b =  1
3


In [51]:
from functools import wraps

# memoize won’t cache unhashable types (dict, lists, etc…) but only the immutable types. Keep that in mind when using it.

def memoize(function):
    memo = {}
    @wraps(function)
    def wrapper(*args):
        if args in memo:
            return memo[args]
        else:
            rv = function(*args)
            memo[args] = rv
            return rv
    return wrapper

@memoize
def fibonacci(n):
    if n < 2: return n
    return fibonacci(n - 1) + fibonacci(n - 2)

fibonacci(25)

75025

In [46]:
# Creating Decorator using Class

import collections
import functools

call_count1 = 0
call_count2 = 0

class memoized(object):
    '''Decorator. Caches a function's return value each time it is called.
       If called later with the same arguments, the cached value is returned
       (not reevaluated).
    '''
    def __init__(self, func):
        self.func = func
        self.cache = {}
    def __call__(self, *args):
        if not isinstance(args, collections.Hashable):
            # uncacheable. a list, for instance.
            # better to not cache than blow up.
            return self.func(*args)
        if args in self.cache:
            return self.cache[args]
        else:
            value = self.func(*args)
            self.cache[args] = value
            return value
    def __repr__(self):
        '''Return the function's docstring.'''
        return self.func.__doc__
    def __get__(self, obj, objtype):
        '''Support instance methods.'''
        return functools.partial(self.__call__, obj)

def fibonacci1(n):
    "Return the nth fibonacci number."
    global call_count1
    call_count1 = call_count1 +1
    if n in (0, 1):
        return n
    return fibonacci1(n-1) + fibonacci1(n-2)
    
@memoized
def fibonacci2(n):
    "Return the nth fibonacci number."
    global call_count2
    call_count2 = call_count2 +1
    if n in (0, 1):
        return n
    return fibonacci2(n-1) + fibonacci2(n-2)

print(fibonacci1(15))
print("call_count1 =", call_count1)
print(fibonacci2(15))
print("call_count2 =", call_count2)


610
call_count1 = 1973
610
call_count2 = 16
